In [6]:
import requests

url = "https://statsapi.mlb.com/api/v1/teams/119/roster?season=2024"
response = requests.get(url)
data = response.json()

In [ ]:
import pybaseball
from pybaseball import statcast

pybaseball.cache.enable()

data = statcast('1968-03-01', '1971-10-30')
data.to_csv('baseball_analysis15.csv', index=False)

In [ ]:
import os
import pandas as pd

data_folder = "../data" 
csv_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.csv')]


df_list = [pd.read_csv(file, na_values=["N/A", "null", ""]) for file in csv_files]
concatenated_df = pd.concat(df_list, ignore_index=True)
concatenated_df.to_csv("analysis_baseball.csv", index=False)

In [6]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pyarrow as pa
import polars as pl

csv_file = "../data/analysis_baseball.csv"
parquet_file = "../data/baseball_analysis.parquet"

BLOCK_SIZE = 120_000_000

read_options = pv.ReadOptions(block_size=BLOCK_SIZE)
csv_reader = pv.open_csv(csv_file, read_options=read_options)

parquet_writer = None

try:
    for i, record_batch in enumerate(csv_reader):
        print(f"Processing batch {i+1} with {record_batch.num_rows} rows...")
        table = pa.Table.from_batches([record_batch])
        if parquet_writer is None:
            parquet_writer = pq.ParquetWriter(
                parquet_file, schema=record_batch.schema, compression="snappy", use_dictionary=True
            )
        parquet_writer.write_table(table)



finally:
    if parquet_writer:
        parquet_writer.close()

print(f"Successfully converted {csv_file} to {parquet_file}")

Processing batch 1 with 326281 rows...
Processing batch 2 with 325599 rows...
Processing batch 3 with 327491 rows...
Processing batch 4 with 327239 rows...
Processing batch 5 with 330639 rows...
Processing batch 6 with 331593 rows...
Processing batch 7 with 332094 rows...
Processing batch 8 with 331220 rows...
Processing batch 9 with 330506 rows...
Processing batch 10 with 327535 rows...
Processing batch 11 with 327859 rows...
Processing batch 12 with 325406 rows...
Processing batch 13 with 325846 rows...
Processing batch 14 with 325949 rows...
Processing batch 15 with 325316 rows...
Processing batch 16 with 325436 rows...
Processing batch 17 with 326256 rows...


ArrowInvalid: In CSV column #37: CSV conversion error to null: invalid value '75.38'

In [ ]:
import pandas as pd
from pyathena import connect

conn = connect(
    s3_staging_dir="s3://calpolyvideo/query-results/",  
    region_name="us-west-1"  
)

In [ ]:
import boto3
from io import BytesIO

s3_bucket = "calpolyvideo"
csv_file_key = "raw_data/analysis_baseball.csv"
parquet_file_key = "processed_data/large_file.parquet"

s3 = boto3.client("s3")

chunk_size = 100000  
parquet_buffer = BytesIO()

for chunk in pd.read_csv(f"s3://{s3_bucket}/{csv_file_key}", chunksize=chunk_size):
    chunk.to_parquet(parquet_buffer, engine="pyarrow", compression="snappy")

s3.put_object(Bucket=s3_bucket, Key=parquet_file_key, Body=parquet_buffer.getvalue())

print("✅ CSV successfully converted to Parquet in chunks.")

In [ ]:
conn = connect(
     s3_staging_dir="s3://calpolyvideo/query-results/",  
     region_name="us-west-1") 
query = "SELECT events FROM my_database.analysis_baseball_csv LIMIT 10;"
df = pd.read_sql(query, conn)

print(df.head())

In [2]:
import duckdb as db

file_path =  "../data/analysis_baseball.csv"

result0 = db.query(f"""SELECT launch_angle
                    FROM '{file_path}'
                    WHERE launch_angle IS NOT NULL
                    ORDER BY game_date ASC;""")
df0 = result0.to_df()
df0

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,launch_angle
0,59.0
1,46.0
2,40.0
3,23.0
4,35.0
...,...
2030613,54.0
2030614,10.0
2030615,-22.0
2030616,7.0
